In [1]:
from src.experiment_based_function import SEEDS
import pandas as pd
%matplotlib inline

In [2]:
import os
import glob


In [3]:
eval_or_grid='eval'

In [8]:
def get_exp8_result(eval_or_grid='eval'):
    res = []    
    for fn in glob.glob('experiment_1001/exp8-spatial/*/%s_res.csv' % eval_or_grid):
        _, seed, _ = fn.split(os.path.sep)
        seed = seed.replace('seed_','')
        df = pd.read_csv(fn, index_col=0)
        df['seed'] = seed
        res.append(df)
    return pd.concat(res)

In [57]:
col_mapping = {' ': ' ',
 'RoadNet': 'RoadNet+Spatial',
 'Segment_NO_TOTAL_~2014+Spatial': 'Social type+Spatial', 
 'Segment_TOTAL_~2014+Spatial': 'Social total+Spatial',
 'RoadNet+Segment_NO_TOTAL_~2014+Spatial': 'RoadNet+Social type+Spatial', 
 'RoadNet+Segment_TOTAL_~2014+Spatial': 'RoadNet+Social total+Spatial'}


In [58]:
eval_res = get_exp8_result('eval')

eval_res.ftr_combo_name = eval_res.ftr_combo_name.apply(lambda x: col_mapping[x])

In [59]:
eval_res.columns, eval_res.shape

(Index(['#ftr_all', '#ftr_keep', '#test_sample', '#train_sample',
        'ftr_combo_name', 'model_name', 'test_acc', 'test_f1_macro',
        'test_f1_micro', 'test_f1_weighted', 'test_mse', 'test_n_classes',
        'train_acc', 'train_f1_macro', 'train_f1_micro', 'train_f1_weighted',
        'train_mse', 'train_n_classes', 'y_dist', 'seed'],
       dtype='object'), (100, 20))

In [60]:
print('train', eval_res.train_n_classes.value_counts().to_dict())
print('test', eval_res.test_n_classes.value_counts().to_dict())


print('train', 'test')
print(eval_res.apply(lambda x: (x.train_n_classes, x.test_n_classes), axis=1).value_counts())

train {5: 100}
test {5: 80, 4: 20}
train test
(5, 5)    80
(5, 4)    20
dtype: int64


In [61]:
f1_choices = ['test_f1_weighted', 'test_f1_macro', 'test_f1_micro']

In [62]:
keys = ['ftr_combo_name', 'model_name']
means = eval_res.groupby(keys).mean()[f1_choices]
stds = eval_res.groupby(keys).std()[f1_choices]


In [69]:
def deliverable_table(table):
    table = table[['RoadNet+Spatial', 'Social total+Spatial', 'Social type+Spatial', 'RoadNet+Social total+Spatial', 'RoadNet+Social type+Spatial']]

    return table

In [71]:
f1_type = f1_choices[2]
table = means.reset_index().pivot(index='model_name', columns='ftr_combo_name', values=f1_type)

dtb = deliverable_table(table)
dtb


ftr_combo_name,RoadNet+Spatial,Social total+Spatial,Social type+Spatial,RoadNet+Social total+Spatial,RoadNet+Social type+Spatial
model_name,,,,,
BAGcls,0.616854,0.559551,0.597753,0.620225,0.613483
XGBcls,0.615730,0.547191,0.594382,0.628090,0.635955
